In [2]:
import risk
from risk.nn import *
from risk.replay_buffer import *
import pickle
import torch
import random
import argparse
from torch.optim import Adam
from torch_geometric.loader import DataLoader


In [3]:
replay_buffer = ReplayBuffer()
replay_buffer.load("replay-buffer/replay_buffer.pkl")
network = Model15()

In [4]:
print(f'Replay Buffer contains: {len(replay_buffer)} experiences')

Replay Buffer contains: 310 experiences


In [5]:
optimizer = Adam(network.parameters(), lr=0.001)
criterion_policy = torch.nn.KLDivLoss(reduction='batchmean')
criterion_value = torch.nn.MSELoss()

In [6]:
replay_buffer.buffer[0]

(StateData(edge_index=[2, 34], map=1, num_nodes=12, num_bonuses=4, num_moves=20, graph_data=[12, 9], global_data=[4], graph_features=[12, 5], graph_edges=[2, 34], bonus_edges=[2, 36], bonus_batch=[12], bonus_nodes=[12], bonus_values=[4], bonus_values_normed=[4], bonus_mapping=[2, 12], income=[1, 2], total_armies=[1, 3], asrcs=[37], adsts=[37], abtch=[37], aarmies=[37], tsrcs=[0], tdsts=[0], tbtch=[0], tarmies=[0], dtgts=[38], dbtch=[38], darmies=[38], attack_data=[37, 20], transfer_data=[0, 19], deploy_data=[38, 10]),
 tensor([0.0610, 0.0610, 0.1010, 0.1010, 0.0610, 0.1610, 0.0010, 0.0610, 0.0010,
         0.0610, 0.0610, 0.0610, 0.0410, 0.0610, 0.0610, 0.0010, 0.0410, 0.0010,
         0.0010, 0.0010]),
 tensor(-0.1160))

In [7]:
states, target_policies, target_values = zip(*replay_buffer.buffer)
print(f"state: {states[0]}")
print(f"move_prob: {target_policies[0]}")
print(f" value: {target_values[0]}")

state: StateData(edge_index=[2, 34], map=1, num_nodes=12, num_bonuses=4, num_moves=20, graph_data=[12, 9], global_data=[4], graph_features=[12, 5], graph_edges=[2, 34], bonus_edges=[2, 36], bonus_batch=[12], bonus_nodes=[12], bonus_values=[4], bonus_values_normed=[4], bonus_mapping=[2, 12], income=[1, 2], total_armies=[1, 3], asrcs=[37], adsts=[37], abtch=[37], aarmies=[37], tsrcs=[0], tdsts=[0], tbtch=[0], tarmies=[0], dtgts=[38], dbtch=[38], darmies=[38], attack_data=[37, 20], transfer_data=[0, 19], deploy_data=[38, 10])
move_prob: tensor([0.0610, 0.0610, 0.1010, 0.1010, 0.0610, 0.1610, 0.0010, 0.0610, 0.0010,
        0.0610, 0.0610, 0.0610, 0.0410, 0.0610, 0.0610, 0.0010, 0.0410, 0.0010,
        0.0010, 0.0010])
 value: -0.11599999666213989


In [8]:
states_data = DataLoader(states[100:200], batch_size=100, shuffle=False)

In [9]:
for batch in states_data:
    v, p = network(batch)

In [10]:
print(f"v: {v}")
print(f"p: {p}")

v: tensor([-0.5041,  0.7710,  0.5836, -0.7473,  0.0175, -0.7667,  0.5593,  0.4668,
         0.3346,  0.9339,  0.3814,  0.1619, -0.2991,  0.3555,  0.1353, -0.1717,
         0.2218,  0.0125, -0.2101,  0.0810,  0.3347, -0.1088,  0.7060,  0.0866,
         0.6156, -0.8412, -0.4749, -0.4030, -0.6543, -0.9697, -0.3187,  0.3951,
        -0.4324,  0.6436,  0.1215,  0.1311, -0.7693,  0.0401,  0.5407,  0.0226,
        -0.4655, -0.0309,  0.1402,  0.4574, -0.9650,  0.3306, -0.9696,  0.2700,
        -0.8951, -0.5116,  0.3083,  0.1377,  0.7842,  0.4298, -0.1069, -0.2120,
         0.4360,  0.7835,  0.6012, -0.3344, -0.6684,  0.2649, -0.1178,  0.4788,
        -0.0376, -0.0935,  0.1584, -0.7534,  0.1718, -0.1916,  0.4937,  0.2629,
         0.3012,  0.2208,  0.5535,  0.1366, -0.8448,  0.1037, -0.5260, -0.9250,
         0.1450, -0.1145, -0.7420, -0.1153, -0.2038, -0.7267, -0.1652, -0.8558,
        -0.0357, -0.0782,  0.7297,  0.8965,  0.0715,  0.7275, -0.5086, -0.3010,
         0.2807, -0.2421, -0.3854,  0